In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import dask.dataframe as dd
from transformers import AutoTokenizer
import pyarrow as pa

In [2]:
X_train = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_train_full.csv')
X_test = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_test_full.csv')

In [3]:
X_train.columns = ['index','review']
X_test.columns = ['index','review']

In [4]:
X_train.set_index(keys='index',inplace=True)
X_test.set_index('index',inplace=True)
X_train.head()

,review
index,
2522958,Doggy Hoots Cracklers: Not as enthusiastic as ...
1160125,great product: Perfect product for freezing mi...
861121,"Awesome Trimmer, Lousy Power: It pains me to g..."
300957,"Oh, T.I. You should be ASHAMED to even call t..."
1610389,This is one of my favorite movies: Forest Gump...


In [5]:
# Load HuggingFace Tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [7]:
# Tokenize reviews
test_inputs = tokenizer(list(X_test['review']),truncation=True,padding=True)

In [8]:
test_inputs.keys()


dict_keys(['input_ids', 'attention_mask'])

In [13]:
# Examining reviews with unknown tokens
has_unknown = []
for i, input in enumerate(test_inputs['input_ids']):
  for tok in input:
    # Searches for an [UNK] token
    if tok == 100:
      # Prints entire line with padding characters removed
      print(f"input {i}: {tokenizer.decode(input).replace(' [PAD]','')}.")
      has_unknown.append(i)
      break

input 34945: [CLS] fat? copy? i think not... : do these look fat to you? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 1do these look like bratz? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! she shows some skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is that horribly wrong? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ( do you show skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is this doll perfect? [UNK]!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [SEP].
input 138420: [CLS] [UNK] [SEP].
input 263693: [CLS] 3. 75 star review ( i'm an engineer..... what can i say, i like precision! ) : - ) \ n \ nback for another run at vapiano's, and another solid effort. the trick is to go early before the banking crowd overruns the place..... so 1130 is ideal. \ n \ ni had the americana pizza, with marinara, fresh mozzerella, pepperoni and italian sausage, and i had them add onions and mushrooms. neopolitan style, not \ " ny \ " style ( with that cru

In [14]:
# Checking the originals of those reviews
for index in has_unknown:
  print(X_test.iloc[index]['review'])

FAT? COPY? I THINK NOT...: Do these look fat to you?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1DO THESE LOOK LIKE BRATZ?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!SHE SHOWS SOME SKIN?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THAT HORRIBLY WRONG?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!(do you show skin?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THIS DOLL PERFECT?YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm1
3.75 Star Review (I'm an Engineer.....what can I say, I like precision!)  :-)\n\nBack for another run at Vapiano's, and another solid effort. The trick is to go early before the Banking crowd overruns the place.....so 1130 i

In [19]:
# Sample from above parenthetical)
untokenizable_sample = 'mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm'
len(untokenizable_sample)




157

It looks like the primary source of unknown tokens are excessively repeated characters, such as in "mmmmmmmmmm..." with 157 m's.

In [20]:
type(test_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [21]:
test_input_ddf = dd.from_dict({'input_ids': test_inputs['input_ids'],
                               'attention_mask': test_inputs['attention_mask']},
                              npartitions=10)

In [22]:
test_input_ddf.head(5)


,input_ids,attention_mask
0,"[101, 2307, 7692, 1024, 2023, 2003, 1037, 2307...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[101, 2054, 2019, 12476, 24385, 999, 1032, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[101, 2204, 2096, 2009, 6354, 1006, 2005, 2026...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[101, 1045, 2293, 2023, 2173, 1012, 1045, 2134...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[101, 2987, 1005, 1056, 2377, 2006, 1037, 4966...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [6]:
schema = pa.schema([('input_ids', pa.list_(pa.int64())),
                    ('attention_mask', pa.list_(pa.int64()))])

In [24]:
test_input_ddf.to_parquet('drive/MyDrive/BERT Sentiment/CSVs/test_inputs.parquet',schema=schema)


In [7]:
# X_train is too big to hold in memory. Going to need to batch them.
batch_size = len(X_train)//15 + 1
batch_size

278857

In [10]:
# Tokenize reviews in batches
for i in range(15):
  start = i*batch_size
  end = min((i+1)*batch_size,len(X_train))
  train_inputs = tokenizer(list(X_train['review'].iloc[start:end]),truncation=True,padding=True)
  train_input_ddf = dd.from_dict({'input_ids': train_inputs['input_ids'],
                                  'attention_mask': train_inputs['attention_mask']},
                                 npartitions=10)
  train_input_ddf.to_parquet(f'drive/MyDrive/BERT Sentiment/CSVs/train_inputs_{i}.parquet',schema=schema)
